In [6]:
using QuantumOptics

const N = 20 # Cavity Dimension

# Cavity Operators
fb = FockBasis(N-1)

a = destroy(fb)
at = create(fb)
Ic = identityoperator(fb)

# qutrit operators
qb = FockBasis(3-1)

Iq = identityoperator(qb)
q = destroy(qb)
qt = create(qb)
sz_ge = Operator(qb,qb, [-1 0 0; 0 1 0; 0 0 1])
sz_gf = Operator(qb,qb, [-1 0 0; 0 0 0; 0 0 1])
sy_ge = Operator(qb,qb, [0 -1im 0; 1im 0 0; 0 0 1])
sy_ef = Operator(qb,qb, [1 0 0; 0 0 -1im; 0 1im 0])
sx_ge = Operator(qb,qb, [0 1 0; 1 0 0; 0 0 1])
sx_ef = Operator(qb,qb, [1 0 0; 0 0 1; 0 1 0])




g = fockstate(qb,0)
e = fockstate(qb,1)
f = fockstate(qb,2)
ge = 1/√2*(g+e)
vac = fockstate(fb,0) 
proj_g = projector(g) ⊗ Ic ⊗ Ic ⊗ Ic
proj_e = projector(e) ⊗ Ic ⊗ Ic ⊗ Ic
proj_f = projector(f) ⊗ Ic ⊗ Ic ⊗ Ic

gvvv = g ⊗ vac ⊗ vac ⊗ vac
evvv = e ⊗ vac ⊗ vac ⊗ vac
gevvv = ((g+e)/√2) ⊗ vac ⊗ vac ⊗ vac
gfvvv = ((g+f)/√2) ⊗ vac ⊗ vac ⊗ vac


######## Hamiltonians ########

const χ = 2π*0.0


H_dispersive_ge = -χ/2 * projector(g) ⊗ ((at*a) ⊗ Ic ⊗ Ic + Ic ⊗ (at*a) ⊗ Ic + Ic ⊗ Ic ⊗ (at*a)) + 
χ/2 * projector(e) ⊗ ((at*a) ⊗ Ic ⊗ Ic + Ic ⊗ (at*a) ⊗ Ic + Ic ⊗ Ic ⊗ (at*a)) +
3*χ/2 * projector(f) ⊗ ((at*a) ⊗ Ic ⊗ Ic + Ic ⊗ (at*a) ⊗ Ic + Ic ⊗ Ic ⊗ (at*a)) # Hamiltonian assuming the drive is omega_c-chi/2

H_dispersive_gf = -χ * projector(g) ⊗ ((at*a) ⊗ Ic ⊗ Ic + Ic ⊗ (at*a) ⊗ Ic + Ic ⊗ Ic ⊗ (at*a)) + 
χ* projector(f) ⊗ ((at*a) ⊗ Ic ⊗ Ic + Ic ⊗ (at*a) ⊗ Ic + Ic ⊗ Ic ⊗ (at*a)) 


# H_dispersive_ef = -χ/2 * sz_gf ⊗ (at*a) ⊗ (at*a) ⊗ (at*a) # dispersive hamiltonian when resonant with the e state. Assuming χ_ge = χ_ef


# Qubit Hamiltonians
function H_qubit_drive_ge(δ, θ = 0)
    # θ = 0 -> rotate around x axis
    # θ = pi/2 -> rotate around y axis 
    # pi flip if ϵ*T_max = pi
    δ/2*(cos(θ)*sx_ge + sin(θ)*sy_ge)
end

function H_qubit_drive_ef(δ, θ = 0)
    # θ = 0 -> rotate around x axis
    # θ = pi/2 -> rotate around y axis 
    # pi flip if ϵ*T_max = pi
    δ/2*(cos(θ)*sx_ef + sin(θ)*sy_ef)
end

function H_cav_drive(ϵ)
    # sign is chosen to match displace 
    # α = ϵ*T_max
    (conj(ϵ * 1im)a + ϵ * 1im*dagger(a)) 
end

function H_CD_ge(β)
    sz_ge⊗(conj(β)*a+β*at) 
end

function H_CD_gf(β)
    sz_gf⊗(conj(β)*a+β*at) 
end


function H_triple_CD_ge(α,β,γ)
    sz_ge⊗(a⊗Ic⊗Ic*conj(α)+at⊗Ic⊗Ic*α + Ic⊗a⊗Ic*conj(β)+Ic⊗at⊗Ic*β + Ic⊗Ic⊗a*conj(γ)+Ic⊗Ic⊗at*γ)
end

function H_triple_CD_gf(α,β,γ)
    sz_gf⊗(a⊗Ic⊗Ic*conj(α)+at⊗Ic⊗Ic*α + Ic⊗a⊗Ic*conj(β)+Ic⊗at⊗Ic*β + Ic⊗Ic⊗a*conj(γ)+Ic⊗Ic⊗at*γ)
end

######### Collapse Operators #########
function photon_loss_cavity_jump_operator(κ = κ_cav,a = a)
    sqrt(κ)*a
end

function photon_loss_qubit_jump_operator(κ = κ_qubit,q = q)
    sqrt(κ)*q
end

# function dephasing_qubit_jump_operator(γ = γ_qubit, a) # need to edit!
#     sqrt(κ)* 0.5*sqrt(γ)*sz
# end

######### Time Evolution #########

### Qubit Rotations
function R_evolution_ge(δ, t,  state, θ = 0)
    H_total = H_qubit_drive_ge(δ, θ) ⊗ Ic ⊗ Ic ⊗ Ic + H_dispersive_ge
    tout, ψt = timeevolution.schroedinger(t, state, H_total);
    last(ψt)
end

function R_evolution_ef(δ, t,  state, θ = 0)
    H_total = H_qubit_drive_ef(δ, θ) ⊗ Ic ⊗ Ic ⊗ Ic + H_dispersive_gf
    tout, ψt = timeevolution.schroedinger(t, state, H_total);
    last(ψt)
end

function R_evolution_ge_loss(δ, t, ρ, J, θ = 0)
    H_total = H_qubit_drive_ge(δ, θ) ⊗ Ic ⊗ Ic ⊗ Ic + H_dispersive_ge
    tout, ρt = timeevolution.master(t, ρ, H_total, J);
    last(ρt)
end

function R_evolution_ef_loss(δ, t, ρ, J, θ = 0)
    H_total = H_qubit_drive_ef(δ, θ) ⊗ Ic ⊗ Ic ⊗ Ic + H_dispersive_gf
    tout, ρt = timeevolution.master(t, ρ, H_total, J);
    last(ρt)
end

### Cavity Displacement
function D_evolution(ϵ ,t , state)
    H_total = Iq ⊗ (H_cav_drive(ϵ[1]) ⊗ Ic ⊗ Ic + Ic ⊗ H_cav_drive(ϵ[2]) ⊗ Ic + 
    Ic ⊗ Ic ⊗ H_cav_drive(ϵ[3])) + H_dispersive_gf
    tout, ψt = timeevolution.schroedinger(t, state, H_total)
    last(ψt)
end

function D_evolution_loss(ϵ ,t , ρ, J)
    H_total = Iq ⊗ (H_cav_drive(ϵ[1]) ⊗ Ic ⊗ Ic + Ic ⊗ H_cav_drive(ϵ[2]) ⊗ Ic + 
    Ic ⊗ Ic ⊗ H_cav_drive(ϵ[3])) + H_dispersive_gf
    tout, ρt = timeevolution.master(t, ρ, H_total, J)
    last(ρt)
end

### Conditinal cavity displacement

function triple_CD_ge_evolution(β ,t , state)
    # final conditinal displacement beta given by β*max(t)
    H_total = H_triple_CD_ge(β[1],β[2],β[3]) 
    tout, ψt = timeevolution.schroedinger(t, state, H_total)
    last(ψt)
end

function triple_CD_gf_evolution(β ,t , state)
    # final conditinal displacement beta given by β*max(t)
    H_total = H_triple_CD_gf(β[1],β[2],β[3])
    tout, ψt = timeevolution.schroedinger(t, state, H_total)
    last(ψt)
end

function triple_CD_ge_evolution_loss(β ,t , ρ, J)
    # final conditinal displacement beta given by β*max(t)
    H_total = H_triple_CD_ge(β[1],β[2],β[3])
    tout, ψt = timeevolution.master(t, ρ, H_total,J)
    last(ψt)
end

function triple_CD_gf_evolution_loss(β ,t , ρ, J)
    # final conditinal displacement beta given by β*max(t)
    H_total = H_triple_CD_gf(β[1],β[2],β[3])
    tout, ψt = timeevolution.master(t, ρ, H_total, J)
    last(ψt)
end

OutOfMemoryError: OutOfMemoryError()

In [2]:
function decode_pauli_string(oper_string, α)
    scale_factor_list = Float64[]
    displacement_list = ComplexF64[]
    is_imag_list = Bool[]

    for character in oper_string
        if character == 'X'
            push!(scale_factor_list,1)
            push!(displacement_list,2*α)
            push!(is_imag_list,false)
        elseif character == 'Y'
            push!(scale_factor_list,1)
            push!(displacement_list,-2*α)
            push!(is_imag_list,true)
        elseif character == 'Z'
            push!(scale_factor_list,1/2*(1/exp((1im*π/(4*α))^2/2)))
            push!(displacement_list,1im*π/(4*α))
            push!(is_imag_list,true)
        elseif character == 'I'
            push!(scale_factor_list,1/2)
            push!(displacement_list,0)
            push!(is_imag_list,false)
        else
            println("Invalid character")
        end
    end
    return scale_factor_list, displacement_list, is_imag_list
end


function measure_3Mode_char_point(ψ0, is_imag_list, displacement_list, t_list_rotation, t_list_conditinal_displacement)
    "is_imag_list: if sum is odd, then the final rotation is around x axis, and we measure the imaginary part of the char function. Otherwise, we measure the real part with a y rotation
    displacement_list: this is the actual value of alpha, -alpha -> implementing waht we understand under an ECD(2alpha) Thats why I divide in the beginning."
    displacement_list = displacement_list./2
    ψ1 = R_evolution_ge(π/2/(last(t_list_rotation)), t_list_rotation, ψ0)
    ψ2 = triple_CD_ge_evolution(displacement_list.*(1/last(t_list_conditinal_displacement)), t_list_conditinal_displacement, ψ1)
    
    if sum(is_imag_list)%2 == 1 # measure imaginary part
        ψ3 = R_evolution_ge(π/2/(last(t_list_rotation)), t_list_rotation, ψ2, π/2)
    else # measure real part
        ψ3 = R_evolution_ge(π/2/(last(t_list_rotation)), t_list_rotation, ψ2, 0)
    end

    return real(expect(sz_ge ⊗ Ic ⊗ Ic ⊗ Ic,ψ3))
end

function measure_3Mode_paulioper(ψ0, pauli_string, t_list_rotation, t_list_conditinal_displacement, α)
    println("entered measure_3Mode_paulioper")
    scale_factor_list, displacement_list, is_imag_list = decode_pauli_string(pauli_string, α)
    
    if sum(is_imag_list)%2 == 0
        println("entered sum(is_imag_list)%2 == 0")
        displacement_list_2 = copy(displacement_list) 
        displacement_list_3 = copy(displacement_list)
        displacement_list_4 = copy(displacement_list)

        displacement_list_2[1] *= -1
        displacement_list_3[2] *= -1
        displacement_list_4[3] *= -1

        cf_point_1 = measure_3Mode_char_point(ψ0, is_imag_list, 
        displacement_list, t_list_rotation, t_list_conditinal_displacement)

        cf_point_2 = measure_3Mode_char_point(ψ0, is_imag_list, displacement_list_2, t_list_rotation, t_list_conditinal_displacement)
        
        cf_point_3 = measure_3Mode_char_point(ψ0, is_imag_list, 
        displacement_list_3, t_list_rotation, t_list_conditinal_displacement)

        cf_point_4 = measure_3Mode_char_point(ψ0, is_imag_list, displacement_list_4, t_list_rotation, t_list_conditinal_displacement)

        # taking care of the 4 cases 000, 110, 101, 011
        return 2*prod(scale_factor_list)*1im^(sum(is_imag_list))*
        (cf_point_1 + (-1)^is_imag_list[1]*cf_point_2 + (-1)^is_imag_list[2]*cf_point_3 + (-1)^is_imag_list[3]*cf_point_4)

    else
            println("entered sum(is_imag_list%2) == 1")
            displacement_list_2 = copy(displacement_list) 
            displacement_list_3 = copy(displacement_list)
            displacement_list_4 = copy(displacement_list)
    
            displacement_list_2[1] *= -1
            displacement_list_3[2] *= -1
            displacement_list_4[3] *= -1
    
            cf_point_1 = measure_3Mode_char_point(ψ0, is_imag_list, 
            displacement_list, t_list_rotation, t_list_conditinal_displacement)
    
            cf_point_2 = measure_3Mode_char_point(ψ0, is_imag_list, displacement_list_2, t_list_rotation, t_list_conditinal_displacement)
            
            cf_point_3 = measure_3Mode_char_point(ψ0, is_imag_list, 
            displacement_list_3, t_list_rotation, t_list_conditinal_displacement)
    
            cf_point_4 = measure_3Mode_char_point(ψ0, is_imag_list, displacement_list_4, t_list_rotation, t_list_conditinal_displacement)
    
           # case 111
            if sum(is_imag_list) == 3 
                return 2*prod(scale_factor_list)*(cf_point_1 + cf_point_2 + cf_point_3 +cf_point_4)
            else
                # case 001, 010, 100
                return 2*prod(scale_factor_list)*(-1)*
                (cf_point_1 + (-1)^is_imag_list[1]*cf_point_2 + (-1)^is_imag_list[2]*cf_point_3 + (-1)^is_imag_list[3]*cf_point_4)
            end
    
        end

end


    

measure_3Mode_paulioper (generic function with 1 method)

In [3]:
tlb = SpinBasis(1//2)

e_tl = spinup(tlb)
g_tl = spindown(tlb)
W_state = 1/√3*(e_tl ⊗ g_tl ⊗ g_tl + g_tl ⊗ e_tl ⊗ g_tl + g_tl ⊗ g_tl ⊗ e_tl)

Iq_tl = identityoperator(tlb)
sx = sigmax(tlb)
sy = sigmay(tlb)
sz = sigmaz(tlb)


function ideal_W_3ModeOper_expect(oper_string)
    tlb = SpinBasis(1//2)
    e_tl = spinup(tlb)
    g_tl = spindown(tlb)
    W_state = 1/√3*(e_tl ⊗ g_tl ⊗ g_tl + g_tl ⊗ e_tl ⊗ g_tl + g_tl ⊗ g_tl ⊗ e_tl)
    oper_dict = Dict('X' => sx, 'Y' => sy, 'Z' => sz, 'I' => Iq_tl)
    three_mode_oper = oper_dict[oper_string[1]] ⊗ oper_dict[oper_string[2]] ⊗ oper_dict[oper_string[3]]

    return real(expect(three_mode_oper, W_state))
end

ideal_W_3ModeOper_expect("XYZ")

# create an array that holds the expecation values of the 64 operators
ideal_W_3ModeOper_expect_array = Float64[]
three_mode_pauli_ops = ["III", "IIX", "IIY", "IIZ", "IXI", "IXX", "IXY", "IXZ", "IYI", "IYX", "IYY", "IYZ", "IZI", "IZX", "IZY", "IZZ", "XII", "XIX", "XIY", "XIZ", "XXI", "XXX", "XXY", "XXZ", "XYI", "XYX", "XYY", "XYZ", "XZI", "XZX", "XZY", "XZZ", "YII", "YIX", "YIY", "YIZ", "YXI", "YXX", "YXY", "YXZ", "YYI", "YYX", "YYY", "YYZ", "YZI", "YZX", "YZY", "YZZ", "ZII", "ZIX", "ZIY", "ZIZ", "ZXI", "ZXX", "ZXY", "ZXZ", "ZYI", "ZYX", "ZYY", "ZYZ", "ZZI", "ZZX", "ZZY", "ZZZ"]

for oper_string in three_mode_pauli_ops
    push!(ideal_W_3ModeOper_expect_array, ideal_W_3ModeOper_expect(oper_string))
end
# print each element of the array on a new line along with its index
println("Ideal W state expectation values")
for (index, value) in enumerate(ideal_W_3ModeOper_expect_array)
    println(three_mode_pauli_ops[index], "  $value")
end



Ideal W state expectation values
III  1.0000000000000002
IIX  0.0
IIY  0.0
IIZ  -0.3333333333333334
IXI  0.0
IXX  0.6666666666666669
IXY  0.0
IXZ  0.0
IYI  0.0
IYX  0.0
IYY  0.6666666666666669
IYZ  0.0
IZI  -0.3333333333333334
IZX  0.0
IZY  0.0
IZZ  -0.3333333333333334
XII  0.0
XIX  0.6666666666666669
XIY  0.0
XIZ  0.0
XXI  0.6666666666666669
XXX  0.0
XXY  0.0
XXZ  -0.6666666666666669
XYI  0.0
XYX  0.0
XYY  0.0
XYZ  0.0
XZI  0.0
XZX  -0.6666666666666669
XZY  0.0
XZZ  0.0
YII  0.0
YIX  0.0
YIY  0.6666666666666669
YIZ  0.0
YXI  0.0
YXX  0.0
YXY  0.0
YXZ  0.0
YYI  0.6666666666666669
YYX  0.0
YYY  0.0
YYZ  -0.6666666666666669
YZI  0.0
YZX  0.0
YZY  -0.6666666666666669
YZZ  0.0
ZII  -0.3333333333333334
ZIX  0.0
ZIY  0.0
ZIZ  -0.3333333333333334
ZXI  0.0
ZXX  -0.6666666666666669
ZXY  0.0
ZXZ  0.0
ZYI  0.0
ZYX  0.0
ZYY  -0.6666666666666669
ZYZ  0.0
ZZI  -0.3333333333333334
ZZX  0.0
ZZY  0.0
ZZZ  1.0000000000000002


In [4]:
function test_exp_measurements(pauli_string, α, t_list_rotation, t_list_conditinal_displacement)
    W = 1/√3*(g ⊗(coherentstate(fb, α) ⊗ coherentstate(fb, -α) ⊗ coherentstate(fb, -α) + coherentstate(fb, -α) ⊗ coherentstate(fb, α) ⊗ coherentstate(fb, -α) + coherentstate(fb, -α) ⊗ coherentstate(fb, -α) ⊗ coherentstate(fb, α)))

    exp_result = measure_3Mode_paulioper(W, pauli_string, t_list_rotation, t_list_conditinal_displacement, α)

    theoretical_result = ideal_W_3ModeOper_expect(pauli_string)

    return exp_result, theoretical_result
end

test_exp_measurements (generic function with 1 method)

In [5]:
t_list_rotation = [0:0.01:0.05;]
t_list_conditinal_displacement = [0:0.1:0.6;]
α = 3.6/2

exp_result, theoretical_result = test_exp_measurements("III", α, t_list_rotation, t_list_conditinal_displacement)

println("Experimental result: ", exp_result)
println("Theoretical result: ", theoretical_result)


entered measure_3Mode_paulioper
entered sum(is_imag_list)%2 == 0

OutOfMemoryError: OutOfMemoryError()